<a href="https://colab.research.google.com/github/venkatrajan247/gemini_api/blob/main/RAG_Pipeline_for_Legal_Query_Resolution_of_india.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph

In [ ]:
!pip install datasets langchain chromadb sentence-transformers

In [ ]:
!pip install langchain-cohere

In [ ]:
import os
from google.colab import userdata
os.environ["cohere_api_key"] = userdata.get('cohere_api_key')
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_cohere import ChatCohere
from langchain.schema.output_parser import StrOutputParser
from langchain.vectorstores import Chroma
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.embeddings import CohereEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_core.runnables import RunnableParallel,RunnablePassthrough

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
!pip install --upgrade datasets

In [ ]:
from datasets import load_dataset
dataset = load_dataset("mratanusarkar/Indian-Laws")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/615 [00:00<?, ?B/s]

indian_law_bare_acts_dataset.parquet:   0%|          | 0.00/16.1M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

In [ ]:
print(text_splitter)
print(dataset)
print(dataset['train'].column_names)


DatasetDict({
    train: Dataset({
        features: ['act_title', 'section', 'law'],
        num_rows: 34244
    })
})
['act_title', 'section', 'law']


In [ ]:
def preprocess_and_split(example):
    # Basic clean up (can add more)
    text = example['text'].replace('\n', ' ').strip()
    chunks = text_splitter.split_text(text)
    return {'chunks': chunks}

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

def preprocess_and_split(example):
    text = example['law'].replace('\n', ' ').strip()
    chunks = text_splitter.split_text(text)
    return {'chunks': chunks}

dataset = dataset.map(preprocess_and_split, batched=False)

all_chunks = []
for item in dataset['train']:
    all_chunks.extend(item['chunks'])

print(f"Total chunks prepared for embedding: {len(all_chunks)}")


Map:   0%|          | 0/34244 [00:00<?, ? examples/s]

Total chunks prepared for embedding: 60381


In [ ]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

embeddings = embedding_model.encode(all_chunks, show_progress_bar=True, batch_size=32)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1887 [00:00<?, ?it/s]

In [ ]:
import chromadb

# Correct: use the new Client method directly with persist_directory
chroma_client = chromadb.PersistentClient(path="./chroma_db")


In [ ]:
import math

# Chroma limit
MAX_BATCH_SIZE = 5000  # safe below 5461

# Helper: batch insert
def batch_insert(collection, all_chunks, embeddings, batch_size=MAX_BATCH_SIZE):
    total = len(all_chunks)
    for i in range(0, total, batch_size):
        batch_chunks = all_chunks[i:i+batch_size]
        batch_embeddings = embeddings[i:i+batch_size]
        batch_ids = [f"chunk_{j}" for j in range(i, i + len(batch_chunks))]
        batch_metadata = [{"source": "indian_laws"} for _ in batch_chunks]

        collection.add(
            documents=batch_chunks,
            embeddings=batch_embeddings,
            metadatas=batch_metadata,
            ids=batch_ids
        )
        print(f"✅ Inserted {i + len(batch_chunks)} / {total} chunks")

# Now run the batch insert
batch_insert(collection, all_chunks, embeddings.tolist())


✅ Inserted 5000 / 60381 chunks
✅ Inserted 10000 / 60381 chunks
✅ Inserted 15000 / 60381 chunks
✅ Inserted 20000 / 60381 chunks
✅ Inserted 25000 / 60381 chunks
✅ Inserted 30000 / 60381 chunks
✅ Inserted 35000 / 60381 chunks
✅ Inserted 40000 / 60381 chunks
✅ Inserted 45000 / 60381 chunks
✅ Inserted 50000 / 60381 chunks
✅ Inserted 55000 / 60381 chunks
✅ Inserted 60000 / 60381 chunks
✅ Inserted 60381 / 60381 chunks


In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

# Use the same embedding model used for storing vectors
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# Reconnect to existing Chroma vector store
vectorstore = Chroma(
    collection_name="indian_laws_collection",
    persist_directory="./chroma_db",  # Must match path used during storage
    embedding_function=embedding_model
)


<ipython-input-22-4b19210e407b>:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
<ipython-input-22-4b19210e407b>:8: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


In [ ]:
# Default retriever with similarity search
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})


In [ ]:
query = "What are the rights of an arrested person under Indian law?"

docs = retriever.get_relevant_documents(query)

for i, doc in enumerate(docs):
    print(f"\nResult {i+1}:")
    print(doc.page_content[:500])  # Print first 500 chars



Result 1:
Constitution of India, 1949  22. Protection against arrest and detention in  certain cases.-  (1) No person who is arrested shall be detained  in custody without being informed, as soon as may be, of the grounds for such  arrest nor shall he be denied the right to consult, and to be defended by, a  legal practitioner of his choice. (2) Every person who is arrested and detained  in custody shall be produced before the nearest magistrate within a period of  twenty-four hours of such arrest excludi

Result 2:
to such condition as to maintenance, discipline and  the punishment of offences and breaches of discipline as the Central Government  may from time to time by order determine. (3) No person shall - (a)  knowingly  assist an internee or a person on parole to escape from custody or the place  set apart for his accidence, or knowingly harbor an escaped internee or person  or parole, or (b)  give an  escaped internee or person on parole any assistance with intent thereby to  p

<ipython-input-24-6999207f8ab8>:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(query)


In [ ]:
from huggingface_hub import login

login()


In [ ]:
from langchain.llms import HuggingFacePipeline
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

model_name = "mistralai/Mistral-7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype="auto")

text_gen_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.7,
    top_k=50,
    top_p=0.95,
    repetition_penalty=1.1
)

llm = HuggingFacePipeline(pipeline=text_gen_pipeline)


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Device set to use cuda:0
<ipython-input-28-c6fec8210909>:20: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=text_gen_pipeline)


In [ ]:
from langchain.prompts import PromptTemplate

template = """You are a legal assistant. Use the following context from Indian law to answer the user's question.

Context:
{context}

Question:
{question}

Answer:"""

prompt = PromptTemplate(template=template, input_variables=["context", "question"])


In [ ]:
from langchain.chains import RetrievalQA

rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",  # options: "stuff", "map_reduce", "refine"
    chain_type_kwargs={"prompt": prompt}
)


In [ ]:
question = "What are the rights of an arrested person under Indian law?"
response = rag_chain.run(question)

print("🔍 Question:", question)
print("🧠 Answer:\n", response)


<ipython-input-31-f0f2c9a69b5c>:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = rag_chain.run(question)
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


🔍 Question: What are the rights of an arrested person under Indian law?
🧠 Answer:
 You are a legal assistant. Use the following context from Indian law to answer the user's question.

Context:
Constitution of India, 1949  22. Protection against arrest and detention in  certain cases.-  (1) No person who is arrested shall be detained  in custody without being informed, as soon as may be, of the grounds for such  arrest nor shall he be denied the right to consult, and to be defended by, a  legal practitioner of his choice. (2) Every person who is arrested and detained  in custody shall be produced before the nearest magistrate within a period of  twenty-four hours of such arrest excluding the time necessary for the journey  from the place of arrest to the court of the magistrate and no such person  shall be detained in custody beyond the said period without the authority of a  magistrate. (3) Nothing in clauses (1) and (2) shall apply  (a) to any person who for the time being is an enemy

In [ ]:
!pip install rank_bm25


In [ ]:
from rank_bm25 import BM25Okapi
import nltk
nltk.download('punkt')

# Prepare BM25 corpus: use the chunks texts
tokenized_corpus = [doc.lower().split() for doc in all_chunks]

bm25 = BM25Okapi(tokenized_corpus)

def bm25_retrieve(query, top_k=5):
    tokenized_query = query.lower().split()
    doc_scores = bm25.get_scores(tokenized_query)
    top_n_indices = doc_scores.argsort()[-top_k:][::-1]
    top_docs = [all_chunks[i] for i in top_n_indices]
    return top_docs


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
dense_docs = retriever.get_relevant_documents(query)


In [ ]:
def hybrid_retrieve(query, top_k=5):
    sparse_docs = bm25_retrieve(query, top_k=top_k)
    dense_docs = retriever.get_relevant_documents(query)

    # Combine and deduplicate preserving order
    seen = set()
    combined_docs = []
    for doc in sparse_docs + dense_docs:
        content = doc.page_content if hasattr(doc, "page_content") else doc
        if content not in seen:
            combined_docs.append(doc)
            seen.add(content)
    return combined_docs[:top_k]


 Re-ranking using Cross-Encoder

In [ ]:
from sentence_transformers import CrossEncoder

cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.66k [00:00<?, ?B/s]

In [ ]:
def rerank(query, docs):
    pairs = [(query, doc.page_content if hasattr(doc, "page_content") else doc) for doc in docs]
    scores = cross_encoder.predict(pairs)
    doc_score_pairs = list(zip(docs, scores))
    # Sort by descending score
    doc_score_pairs.sort(key=lambda x: x[1], reverse=True)
    reranked_docs = [doc for doc, score in doc_score_pairs]
    return reranked_docs


Full Hybrid Retrieval + Re-ranking pipeline

In [ ]:
def hybrid_rerank_pipeline(query, top_k=5):
    # Step 1: hybrid retrieve
    combined_docs = hybrid_retrieve(query, top_k=top_k*3)  # get more for rerank

    # Step 2: rerank
    reranked_docs = rerank(query, combined_docs)

    # Step 3: return top_k reranked
    return reranked_docs[:top_k]


In [ ]:
from langchain.schema import BaseRetriever
from langchain.docstore.document import Document
from typing import List
from pydantic import Field

class HybridRetriever(BaseRetriever):
    top_k: int = Field(default=5)

    def get_relevant_documents(self, query: str) -> List[Document]:
        docs = hybrid_rerank_pipeline(query, top_k=self.top_k)

        # Wrap in Document objects if needed
        wrapped_docs = []
        for d in docs:
            if isinstance(d, Document):
                wrapped_docs.append(d)
            else:
                wrapped_docs.append(Document(page_content=d))
        return wrapped_docs


<ipython-input-44-03c542d69561>:6: DeprecationWarning: Retrievers must implement abstract `_get_relevant_documents` method instead of `get_relevant_documents`
  class HybridRetriever(BaseRetriever):


In [ ]:
hybrid_retriever = HybridRetriever(top_k=5)


In [ ]:
rag_chain_hybrid = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=hybrid_retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt}
)


In [ ]:
query = "What is the procedure for filing a writ petition in Indian courts?"

answer = rag_chain_hybrid.run(query)
print("Answer:\n", answer)


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Answer:
 You are a legal assistant. Use the following context from Indian law to answer the user's question.

Context:
Indian Christian Marriage Act,1872  46. Petition when Marriage Registrar  refuses certificate Whenever a Marriage  Registrar refuses to issue a certificate under this Part, either of the parties  intending marriage may apply by petition, where the district of such Registrar  is within any of the towns of Calcutta, Madras and Bombay, to a Judge of the  High Court, or if such district is not within any of the said towns, then to  the District Judge. Procedure on petition The said Judge of the  High Court, or district Judge, as the case may be, may examine the allegations  of the petition in a summary way, and shall decide thereon. The decision of such  Judge of the High Court or district Judge, as the case may be, shall be final,  and the Marriage Registrar to whom the application for the issue of a  certificate was originally made shall proceed in accordance therewith.


In [ ]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory


In [ ]:
memory = ConversationBufferMemory(
    memory_key="chat_history",  # required key for ConversationalRetrievalChain
    return_messages=True
)


<ipython-input-49-0ef681c1b595>:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [ ]:
conversational_rag = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=hybrid_retriever,
    memory=memory,
    chain_type="stuff",  # optional
    combine_docs_chain_kwargs={"prompt": prompt}  # optional custom prompt
)


In [ ]:
chat_history = []

query1 = "What is the right to equality?"
response1 = conversational_rag.run(query1)
print("Bot:", response1)

query2 = "Does it apply to employment discrimination?"
response2 = conversational_rag.run(query2)
print("Bot:", response2)


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/pyth

Bot: You are a legal assistant. Use the following context from Indian law to answer the user's question.

Context:
Right to Fair Compensation and Transparency in Land Acquisition, Rehabilitation and Resettlement Act, 2013  72.  Costs. 1.         Every  such award shall also state the amount of costs incurred in the proceeding  under this Chapter, and by what persons and in what proportions they are to be  paid. 2.         When  the award of the Collector is not upheld, the cost shall ordinarily be paid by  the Collector, unless the Authority concerned is of the opinion that the claim  of the applicant was so extravagant or that he was so negligent in putting his  case before the Collector that some deduction from his costs should be made or  that he should pay a part of the Collector’s

Transfer of Property Act, 1882 6. What may be transferred Property of any kind may be transferred, except as otherwise  provided by this Act or by any other law for the time being in force. (a) The chan

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Bot: You are a legal assistant. Use the following context from Indian law to answer the user's question.

Context:
Transfer of Property Act, 1882 6. What may be transferred Property of any kind may be transferred, except as otherwise  provided by this Act or by any other law for the time being in force. (a) The chance of an heir-apparent succeeding to an estate, the  chance of a relation obtaining a legacy on the death of a kinsman, or any other  mere possibility of a like nature, cannot be transferred. (b) A mere right of re-entry for breach of a condition  subsequent cannot be transferred to anyone except the owner of the property  affected thereby. (c) An easement cannot be transferred apart from the dominant  heritage. (d) An interest in property restricted in its enjoyment to the  owner personally cannot be transferred by him. (dd) A right to future maintenance, in whatsoever manner  arising, secured or determined, cannot be transferred. (e) A mere right to sue cannot be transferr